## Deliverable 2. Create a Customer Travel Destinations Map.

In [35]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_database/weatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Te Anau,NZ,-45.4167,167.7167,49.73,77,100,5.88,overcast clouds
1,1,Nizwa,OM,22.9333,57.5333,87.39,18,7,1.70,clear sky
2,2,Jamestown,US,42.0970,-79.2353,63.99,83,75,3.00,broken clouds
3,3,Albany,US,42.6001,-73.9662,69.03,88,88,3.00,overcast clouds
4,4,Slave Lake,CA,55.2834,-114.7690,58.98,55,90,13.80,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                         (city_data_df["Max Temp"] <= max_temp)]

In [6]:
# 4a. Determine if there are any empty rows.
new_df.count()

Unnamed: 0             210
City                   210
Country                210
Lat                    210
Lon                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Weather description    210
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_df[["City", "Country", "Max Temp", "Weather description", "Lat", "Lon"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather description,Lat,Lon,Hotel Name
1,Nizwa,OM,87.39,clear sky,22.9333,57.5333,
8,Bigaa,PH,87.13,broken clouds,13.1978,123.7578,
9,Jeremie,HT,80.44,scattered clouds,18.6500,-74.1167,
14,West Bay,KY,82.00,few clouds,19.3667,-81.4167,
21,Avarua,CK,80.65,light rain,-21.2078,-159.7750,
25,Sola,VU,77.38,overcast clouds,-13.8833,167.5500,
29,Kapaa,US,82.11,scattered clouds,22.0752,-159.3190,
33,Pochutla,MX,84.02,overcast clouds,15.7432,-96.4661,
36,Fare,PF,80.89,few clouds,-16.7000,-151.0167,
40,Cockburn Town,TC,79.16,clear sky,21.4612,-71.1419,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "key":g_key,
    "radius": 5000,
    "type": "lodging",
}

# 6b. Iterate through the hotel DataFrame.
for index,rows in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = rows["Lat"]
    lng = rows["Lon"]
    params["location"]=f"{lat},{lng}"
    
#     # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#   print(base_url)


#     # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
#     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except Exception as Error:
        print(Error)


list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [28]:
hotel_df

,City,Country,Max Temp,Weather description,Lat,Lon,Hotel Name
1,Nizwa,OM,87.39,clear sky,22.9333,57.5333,Orient Apartments
8,Bigaa,PH,87.13,broken clouds,13.1978,123.7578,Miña's Mayon View Resort
9,Jeremie,HT,80.44,scattered clouds,18.6500,-74.1167,Auberge’Inn
14,West Bay,KY,82.00,few clouds,19.3667,-81.4167,Calypso Cove
21,Avarua,CK,80.65,light rain,-21.2078,-159.7750,Paradise Inn
...,...,...,...,...,...,...,...
662,Morro Bay,US,75.74,few clouds,35.3658,-120.8499,Inn at Morro Bay
663,Dalbandin,PK,89.08,clear sky,28.8947,64.4101,Sanjrani House
664,Obra,IN,76.73,overcast clouds,24.4167,82.9833,Hotel Vishal
669,Dhaka,BD,82.38,broken clouds,23.7104,90.4074,Main Hostel SSMC


In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how = "any")

In [29]:
# 8a. Create the output File (CSV)
output_data_file = ("weatherPy_vacation.csv")

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>Hotel Name</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Weather description}</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lon"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lon"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))